In [ ]:
import gym, math, glob
import numpy as np

import cv2

from timeit import default_timer as timer
from datetime import timedelta

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

# from utils.wrappers import *
# from agents.DQN import Model as DQN_Agent
# from utils.ReplayMemory import ExperienceReplayMemory

from utils.hyperparameters import Config
from utils.plot import plot_all_data
import Game.tetris_fun as game

In [ ]:
# Solution for error: no available video device
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [ ]:
class DQN(nn.Model):
    def __init__(self, num_actions, in_channels=3):
        super(DQN, self).__init__()
        self.in_channels = in_channels
        self.num_actions = num_actions
        
        # could add batchnorm2d layers after each covnet if data volume is too large
        # see: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
        
        self.conv1 = nn.Conv2d(self.in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        self.fc1 = nn.Linear(self.feature_size(), 512)
        self.fc2 = nn.Linear(512, self.num_actions)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        # reshape the tensor to one dimension for fc layers
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))

        return self.fc2(x)
    
    def feature_size(self):
        return self.conv3(self.conv2(self.conv1(torch.zeros(1, *self.in_channels)))).view(1, -1).size(1)
#         return (size - (kernel_size - 1) - 1) // stride  + 1
    